# Setup

In [1]:
import pandas as pd

from os import listdir
from os.path import isfile, join

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
import spacy
import re

# INSTALL FIRST (noted again when needed below):
# nltk.download('stopwords')
# python -m spacy download en_core_web_sm


In [4]:
# Raw data in, out data goes to main data folder
DATA_PATH = '../data/raw/'
OUTPUT_PATH = '../data/'

In [5]:
# Check jsons in data raw folder
datafiles = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f))]

print('Index, Filename')
print(list(zip([index for index, value in enumerate(datafiles)], datafiles)))

Index, Filename
[(0, 'deepfake_txt.csv'), (1, 'arxiv_disinformation.csv'), (2, 'arxiv_deepfake.csv'), (3, 'results.csv'), (4, 'reddit_machinelearning.csv'), (5, 'test_df.csv'), (6, 'mit_data.csv'), (7, 'importai_data.csv'), (8, 'cna_CNA_data.csv'), (9, 'synced_data.csv'), (10, 'ieee_data.csv'), (11, 'test_compiled.csv')]


In [ ]:
# Load df
load_file = datafiles[-1]

df = pd.read_json(DATA_PATH + load_file, convert_dates=True, lines=True, orient='records')

In [7]:
# to process existing csv
csvfiles = [f for f in listdir(OUTPUT_PATH) if isfile(join(OUTPUT_PATH, f))]
print('Index, Filename')
print(list(zip([index for index, value in enumerate(csvfiles)], csvfiles)))

Index, Filename
[(0, 'deepfake_txt.csv'), (1, 'arxiv_disinformation.csv'), (2, 'arxiv_deepfake.csv'), (3, 'results.csv'), (4, 'reddit_machinelearning.csv'), (5, 'test_df.csv'), (6, 'mit_data.csv'), (7, 'importai_data.csv'), (8, 'cna_CNA_data.csv'), (9, 'synced_data.csv'), (10, 'ieee_data.csv'), (11, 'test_compiled.csv')]


In [8]:
# Load df
load_file = csvfiles[-1]

CONVERTERS = {'tags': eval, 'arxiv_primary_category': eval,"published_parsed": eval}

df = pd.read_csv(OUTPUT_PATH + load_file, converters=CONVERTERS)

In [11]:
df.tail(20)

,title,url,date,text,cleaning,tokens,summary
37,"Issue 7: January 27, 2022 [ web | pdf ]",https://www.cna.org/centers/cna/cip/china/chin...,2022-01-27,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,The China AI and Autonomy Report | CNA Contac...,"['ai', 'contact', 'direction', 'toggle', 'navi...",NaN
38,"Issue 8: February 10, 2022 [ web | pdf ]",https://www.cna.org/centers/cna/cip/china/chin...,2022-02-10,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,The China AI and Autonomy Report | CNA Contac...,"['ai', 'contact', 'direction', 'toggle', 'navi...",NaN
39,"Issue 9: February 24, 2022 [ web | pdf ]",https://www.cna.org/centers/cna/cip/china/chin...,2022-02-24,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,The China AI and Autonomy Report | CNA Contac...,"['ai', 'contact', 'direction', 'toggle', 'navi...",NaN
40,"Issue 10: March 10, 2022 [ web | pdf ]",https://www.cna.org/centers/cna/cip/china/chin...,2022-03-10,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,The China AI and Autonomy Report | CNA Contac...,"['ai', 'contact', 'direction', 'toggle', 'navi...",NaN
41,"Issue 11: March 24, 2022 [ web | pdf ]",https://www.cna.org/centers/cna/cip/china/chin...,2022-03-24,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,The China AI and Autonomy Report | CNA Contac...,"['ai', 'contact', 'direction', 'toggle', 'navi...",NaN
42,"Issue 12: April 7, 2022 [ web | pdf ]",https://www.cna.org/centers/cna/cip/china/chin...,2022-04-07,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,The China AI and Autonomy Report | CNA Contac...,"['ai', 'contact', 'direction', 'toggle', 'navi...",NaN
43,"Issue 13: April 21, 2022 [ web | pdf ]",https://www.cna.org/centers/cna/cip/china/chin...,2022-04-21,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,The China AI and Autonomy Report | CNA Contac...,"['ai', 'contact', 'direction', 'toggle', 'navi...",NaN
44,Meta has built a massive new language AI—and i...,https://www.technologyreview.com/2022/05/03/10...,2022-05-03,Meta’s AI lab has created a massive new langua...,Meta’s AI lab has created a massive new langua...,"['ai', 'lab', 'create', 'massive', 'new', 'lan...",Facebook’s parent company is inviting research...
45,"The Download: Language-preserving AI, and hack...",https://www.technologyreview.com/2022/04/22/10...,2022-04-22,"This is today's edition of The Download, our w...","This is today's edition of The Download, our w...","['today', 'edition', 'download', 'weekday', 'n...",Plus: tweeting out Silicon Valley's secrets</p>\n
46,A new vision of artificial intelligence for th...,https://www.technologyreview.com/2022/04/22/10...,2022-04-22,This story is the fourth and final part of MIT...,This story is the fourth and final part of MIT...,"['story', 'fourth', 'final', 'part', 'mit', 't...","ln a remote rural town in New Zealand, an Indi..."


In [ ]:
df.columns

# Processing

In [ ]:
# INSTALL FIRST: python -m spacy download en_core_web_sm
# Initialize spacy 'en' model
nlp = spacy.load("en_core_web_sm") # disable=['parser', 'ner']) ## if you need efficiency


In [ ]:
# Column name where you want to do stuff
action_col = 'summary'

### Initial cleaning

In [ ]:
def cleaning(text):
    cleaned_text = text.lower() # lower case
    cleaned_text = cleaned_text.replace("\n", " ")
    cleaned_text = cleaned_text.replace("\t", " ")
    cleaned_text = cleaned_text.replace('\r', '')

    cleaned_text = re.sub('[^\S\r\n]{2,}', ' ', cleaned_text) # extra spaces
    cleaned_text = cleaned_text.rstrip()
    return cleaned_text

In [ ]:
df['cleaning'] = df[action_col].dropna().apply(lambda x: cleaning(x))
df.cleaning

### Remove punctuation

In [ ]:
# Creat punctuation list

special_punctuation = '：，,《。》“„:一・«»”“]'

final_punctuation = string.punctuation + special_punctuation
final_punctuation

In [ ]:
def remove_punctuation(txt):
    txt_nopunct = ''.join([c for c in txt if c not in final_punctuation])
    return txt_nopunct

In [ ]:
df['cleaning'] = df['cleaning'].dropna().apply(lambda x: remove_punctuation(x))
df.cleaning

### Tokenize, lemmatize, drop POSs

In [ ]:
# Words to keep even if they are not in POS
to_keep = ["GAN", "GPT-3", "fake news", "disinformation"]

In [ ]:
# Lemmatization function

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    doc = nlp("".join(texts)) #nlp(sent) #" ".join(sent)) 
    texts_out = [token.lemma_ for token in doc if token.pos_ in allowed_postags or token.text in to_keep]
    return texts_out

In [ ]:
df['lemma_text'] = df['cleaning'].dropna().apply(lambda x:  lemmatization(x, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']))


In [ ]:
df['lemma_text']

In [ ]:
# Not needed, unless no lemmaization is done
#df['tokens'] = df.dropna().apply(lambda row: nltk.word_tokenize(row['tokens']), axis=1)

### Remove stopwords

In [ ]:
# Creat stopword list

# RUN
# nltk.download('stopwords')

lang = 'english'

stop_words = list(stopwords.words(lang))

# add in anything else we need to remove, eg. for some analysis, any search tags would be dropped from text
new_stop_words = [] 

final_stop_words = stop_words + new_stop_words

In [ ]:
def remove_stopwords(txt):
    txt_nostops = [w for w in txt if not w in final_stop_words]
    #txt_nostops = ' '.join([w for w in txt if not w in stop_words]) # Alternate
    return txt_nostops

In [ ]:
df['tokens'] = df['lemma_text'].dropna().apply(lambda x:  remove_stopwords(x))
df['tokens']

In [ ]:
# Optional functions

def remove_numbers(txt):
    result = ''.join([i for i in txt if not i.isdigit()])    
    return result

def get_numbers(txt):
    x = re.findall(r'\d+', txt)
    return len(x)

def pos_count(pos, txt):
    x = [token for token in txt if token.endswith(pos)]
    #y = [token.split('/')[0] for token in x] # use when I need lists with just these!
    return len(x)
    #return y


# Get Tags

In [ ]:
df.tags[1]

In [ ]:
def get_tag(x):
      tag = x[0]['term']
      # TO DO - scrape https://arxiv.org/category_taxonomy to translate codes to plain english
      return tag

In [ ]:
df['category'] = df['tags'].dropna().apply(lambda x:  get_tag(x))


# Get time periods

In [ ]:
def get_period(x, period):
      #output = x.split(',')[period] ## For string splitting
      output = x[:period]
      if len(output) == 1:
            return output[0]
      else:
            return str(output[0]) + '-' + str(output[1])

In [ ]:
df['year'] = df['published_parsed'].dropna().apply(lambda x:  get_period(x, 1))
df['month_year'] = df['published_parsed'].dropna().apply(lambda x:  get_period(x, 2))



# Check output

In [ ]:
df.head(2)

In [ ]:
df.columns

In [ ]:
df.drop('lemma_text', axis=1, inplace=True)
#df.drop('cleaning', axis=1, inplace=True)

In [ ]:
out_file = load_file.split('.')[0]

In [ ]:
OUTPUT_PATH

In [ ]:
out_file

In [ ]:
df.to_csv(OUTPUT_PATH + out_file + '.csv', index=False)

In [3]:
test = """The China AI and Autonomy Report | CNA


    
        
		        
          
            
                
					
							
	
	
	
	


					

					
							Contact Us
	Directions


					

					
												
                        						
												
						
							
						

						
							
						
						
                    

					

                
      
            

            			
				 
                    					
                

				
					
				
				
				
				
			
            
        

        
		        
        
            
                
                      
                          Toggle navigation
                          
                          
                          
                    
                
            
                
                    	About Us
	Our Experts
	Our Research
	
                            Centers and Divisions 
                            	Center for Naval Analyses
	Institute for Public Research


                        
	
                            News 
                            	News and Information
	InDepth


                        
	
                            Careers 
                            	Apply Now
	Careers At CNA
	What We Offer
	Benefits
	Research Internship Program
	Diversity
	Community


                        
	Video
	
                            Podcasts 
                            	CNA Talks
	AI with AI
	Coming in from the Cold
	Justice Talks


                        


                

            

        

        
        
            
                
                    
                        
                        
                            Center for Naval Analyses > 
                                China and Indo-Pacific Security Affairs > China Studies

                         The China AI and Autonomy Report

                         
                             
                            A biweekly newsletter on AI and autonomy developments in China
                            
                            

                        

                        
                            
                                Issue 11, March 24, 2022 [Download PDF Version]

    
                                Welcome to the China AI and Autonomy Report, a biweekly newsletter published by CNA. 

                                
                                As the war in Ukraine continues, rumors have been circulating about the role of PRC-made drones in the conflict. In this issue we note two of them—allegations that Russia has asked the PRC to supply drones, and accusations that a leading PRC-based commercial drone manufacturer has been limiting the technical capabilities of drones used by the Ukrainian military in order to provide an advantage to the Russian armed forces. Meanwhile, PRC media outlets have been busy reporting the highlights from this year’s National People’s Congress and Chinese People's Political Consultative Conference meetings (known as the “two sessions”), including statements and proposals related to AI made by China’s political and tech leaders. Policy suggestions by delegates included advocating for a more favorable policy environment for autonomous vehicles, the development of “green AI,” government leadership in the creation of China’s metaverse, and the creation of a legal framework for UAVs in China. As expected, PRC premier Li Keqiang discussed AI in the context of the development of the PRC’s digital economy. We also cover a UAV built by a PRC research institute that reportedly set the record in China for the longest endurance capability and note that, for the first time in a phase 3 clinical trial in China, an AI chip was successfully implanted into an epilepsy patient's brain. 

                            
                                
                                    PRC DRONES AND THE RUSSIA-UKRAINE WAR
                                

                                
                                
                                
                                    Russia reportedly asked the PRC for drones. On March 14, CBS News
                                    
                                        reported
                                    
                                     that, according to an unnamed US official, Russia has asked the PRC to
                                    supply drones to its military. The report came amidst US government
                                    accusations that Russia has asked the PRC for economic and military support
                                    for its invasion of Ukraine. The PRC government has
                                    
                                        denied reports
                                    
                                    of the Russian request for military and economic assistance and of the
                                    PRC's willingness to provide such support, calling them ""malicious
                                    disinformation.""
                                

                                
                                    Leading PRC drone manufacture accused of limiting the capabilities of UAVs
                                    used by the Ukrainian military in order to provide Russian forces military
                                    advantage. In the ongoing war in Ukraine, both Russia and Ukraine are
                                reportedly using drones made by Shenzhen-based	DJI, a world leader in commercial drone
                                    production. Citing sources inside Ukraine, on March 10 Volodymyr
                                Shymanskyy, the co-founder of	Blynk IoT, a software company with an
                                    office in Kyiv,
                                    
                                        accused DJI
                                    
                                    of limiting the capabilities of its AeroScope technology for the Ukrainian
                                    army. According to DJI,
                                    
                                        AeroScope technology
                                    
                                    ""is able to identify the vast majority of popular drones on the market
                                    today by monitoring and analyzing their electronic signals to gain critical
                                    information.""
                                

                                
                                    Shymanskyy claimed that because DJI has allegedly limited AeroScope for the
                                    Ukrainian army, the Russian military has the technical capabilities to
                                    track Ukrainian DJI drone operators, while the Ukrainian army cannot track
                                    Russian drone operators, giving ""a significant air reconnaissance edge to
                                    Russian invaders."" DJI North America responded to these claims by stating
                                    that they were
                                    
                                        false
                                    , and further explained that DJI is ""aware of the problems with some
                                    AeroScope units in Ukraine"" and attributed the issue to ""prolonged loss of
                                    power/internet"" rather than ""deliberate action to downgrade AeroScope"" in
                                    Ukraine.
                                

                                Two Sessions

                                
                                    From March 4-11, the PRC held its ""two sessions""-an annual event during
                                    which the National People's Congress (NPC) and the Chinese People's
                                    Political Consultative Conference (CPPCC) conduct concurrent sessions. At
                                    the meetings, thousands of delegates discuss a wide array of topics related
                                    to the PRC's economic development, governance, and international relations.
                                    Selected tech executives attend the NPC and CPPCC sessions-the latter body
                                    being
                                    
                                        intentionally designed to include delegates
                                    
                                    from a diversity of social and political strata in the PRC as part of the
                                    Chinese Communist Party's (CCP) ""united front"" work. Although major
                                    political decisions in the PRC are generally made during high-level
                                    meetings of the CCP in the fall preceding the two sessions, the highly
                                    scripted meetings are often viewed as a way to ""take the temperature"" of
                                    the PRC's political class.
                                

                                
                                    Premier Li Keqiang discusses AI in the context of the digital economy in
                                    PRC government report. In line with previous iterations of the ""two
                                    sessions,"" PRC Premier Li Keqiang delivered the annual government work
                                report (see full text from PRC government website in Chinese	here
                                    and from Xinhua in English
                                    
                                        here
                                    ). The work report typically covers a broad range of government
                                    achievements over the previous year and goals for the future. Although an
                                    in-depth discussion on AI was not expected in this speech, Premier Li
                                    mentioned AI in relation to the development of the PRC's digital economy.
                                    He advocated for the development of ""smart cities and digital villages"" and
                                    the expansion of digital industries, such as AI. He further discussed smart
                                    manufacturing as an important factor in enhancing the core competitiveness
                                    of the PRC's manufacturing industry and encouraged local government
                                    spending on ""green and smart home appliances"" in rural areas to stimulate
                                    domestic consumption.
                                

                                
                                    Of note, the other anticipated work report of the ""two sessions,"" the NPC
                                Work Report (see in Chinese	
                                here), 
                                discussed the importance of science and technology innovation, but did
                                    not make specific reference to AI technologies, only broadly stating the
                                    importance of strengthening the ""digitization, informatization, and
                                    intelligentization"" for aspects related to the NPC's own work.
                                

                                
                                    Political and tech leaders make policy suggestions for autonomous vehicles,
                                    green AI, the metaverse, and drones. Some suggestions made by CPCC
                                    delegates include:
                                

                                
                                    A more favorable policy environment for autonomous vehicles: 
                                    
                                    
                                        According to the Shanghai-based Observer
                                    , Baidu CEO
                                    
                                        Robin Li, 
                                    who attended the CPPCC, suggested removing bottlenecks in the policy
                                    environment for autonomous vehicles with a three-part plan that would: 1)
                                    allow driverless vehicles to operate on the road without safety officers
                                    and create pilot areas that allow driverless vehicles to carry passengers;
                                    2) revise and implement the Road Traffic Safety Law to set a legal
                                    foundation for driverless vehicles; and 3) build an intelligent
                                    transportation infrastructure, take full advantage of remote control
                                    capabilities enabled by 5G, and improve traffic efficiency and safety
                                    through vehicle-road coordination.
                                

                                
                                        Procuring energy-efficient chips to enable Green AI: 
                                    
                                        According to a statement provided to the Global Times
                                    , Baidu CEO Robin Li called for an ""accelerated push"" toward ""green AI,""
                                    emphasizing the need for energy-efficient AI chips. Green AI refers to the
                                    ""use of green power and algorithms to enable improved energy efficiency and
                                    reduced carbon emissions from data centers."" To realize green AI, Li stated
                                    that businesses would need to be guided toward research and procurement of
                                    energy-efficient AI chips, thereby aligning better with green algorithms.
                                

                                
                                        
                                            Advocating for central government leadership over the ""Chinese
                                            metaverse"":
                                        
                                    
                                    
                                        According to the South China Morning Post
                                    , Liu Wei, a CPPCC delegate and president of the Guangzhou-based AI
                                    services company
                                    
                                        PCI Technology Group
                                    , stated that ""the government should lead the creation of the 'Chinese
                                    metaverse' digital economy."" Liu added that the metaverse could help
                                    digitize industries hurt by COVID-19, such as tourism, and assist with the
                                    rollout of the PRC's
                                    digital yuan. Although some delegates expressed optimism about the metaverse, others
                                    cautioned about the risks it could bring. NPC delegate
                                    
                                        Tencent CEO Ma Huateng
                                    
                                    warned that the rise of the metaverse, non-fungible tokens, and Web3 ""have
                                    brought new risks to finance, technology and social governance at a time
                                    when the virtual and physical worlds are merging.""
                                

                                Proposing a law to govern UAVs in China:
                                    
                                        According to Shanghai-based The Paper
                                    
                                    , Shao Zhiqing, delegate to the NPC and member of the Party Central
                                    Committee of the China Zhi Gong Party (China Public Interest Party),
                                    
                                        one of PRC's eight minor political parties
                                    , proposed a UAV flight management law. Currently no law exists that
                                    addresses drone flight issues in the PRC, although in 2018 the ""Interim
                                    Regulations on the Management of Unmanned Aircraft Flights"" was released
                                    for comment. Shao's proposal for the law included several suggestions,
                                    including clarifying the responsibilities of relevant government
                                    departments regarding drone flight operations and the drone industry;
                                    refined classification, supervision, and certification of drones on the
                                    market; registration of civilian drones that exceed a certain weight with
                                    public security; an application and approval system for drone flight to
                                    ensure airspace safety; and requiring the certification and training of
                                    owners flying drones over a certain weight.
                                

                                Unmanned Systems

                                
                                    New UAV sets a record for ""the longest endurance of its kind in China.""
                                    
                                        China Daily
                                    
                                    reported that the Qing Ou 30 UAV, or ""Blue Seagull 30,"" the first
                                    PRC-developed fixed wing drone capable of vertical takeoff and landing,
                                    made its maiden flight in February. The UAV can fly up to 800 kilometers in
                                    nine hours at speeds of up to 90 km/hour and has a maximum takeoff weight
                                    of 30 kilograms. The Qing Ou 30 was built by the Chongqing Research
                                    Institute, a
                                    
                                        collaboration
                                    
                                between China's elite	Harbin Institute of Technology and an
                                    administrative district of the Chongqing municipal government. The director
                                    of the Chongqing Research Institute, Qin Jiang, emphasized that the use of
                                    hydrogen-powered batteries instead of traditional lithium batteries helps
                                    contribute to the endurance of the UAV. A video of the maiden flight of the
                                UAV, held in January in Chongqing, is available on	Xinhua News.
                                

                                
                                    Technology firms from China and Saudi Arabia announce a joint venture,
                                    Aerial Solutions, and cooperation agreements to produce UAVs. PRC
                                    state-owned enterprise and leading military electronics manufacturer China
                                    Electronics Technology Group Corporation (CETC) and Saudi Arabia-based
                                    company Advanced Communications and Electronics Systems (ACES)
                                    
                                        signed an agreement
                                    
                                to manufacture UAVs through a joint venture during the	World Defense Show, held in
                                    Riyadh from 6-9 March. 
                                    
                                        The joint venture, called Aerial Solutions, will manufacture UAV
                                        systems that ""include communications, flight-control, camera, radar and
                                        wireless-detection systems."" Aerial Solutions will also include a
                                        CETC-sponsored research and development center. CCP-affiliated news
                                        website China Xiaokang also reported that PRC-based Zhongshan
                                    
                                    
                                        Fukun Aviation Technology Company
                                    
                                    -a company founded in 2018 that focuses on UAVs, flight control systems,
                                    and AI products-signed a ""strategic cooperation agreement"" to carry out
                                    in-depth cooperation in the field of industrial drones with both ACES and
                                    Aerial Solutions.
                                

                                Industry

                                
                                    Hikvision is reportedly seeking a second-hand ASML machine for chip
                                    production. According to the
                                    
                                        South China Morning Post
                                    
                                    , surveillance camera giant and
                                    
                                        PRC-designated national ""AI champion""
                                    
                                    Hangzhou Hikvision Digital Technology is seeking to purchase a second-hand
                                    lithography machine for computer chip production known as extreme
                                    ultraviolet lithography (EUV) made by the Dutch firm ASML Holdings.
                                    According to the report, ""Hikvision's pursuit of a second-hand ASML machine
                                    to kick-start its own chip production reflects lingering uncertainties in
                                    the global semiconductor supply chain amid new Covid-19 outbreaks in China,
                                    heightened tensions between Beijing and Washington, and the potential
                                    impact of the war in Ukraine on international trade.""
                                

                                
                                    ASML is Europe's largest technology company by market capitalization and is
                                    the global leader in lithography systems for the production of advanced
                                    computer chips, reportedly occupying 80 to 85 percent of the market. When
                                    it comes to EUV-the most advanced type of chipmaking lithography
                                    machine-ASML's market share is 100 percent. Currently, only a few
                                    companies, including Intel, Samsung, and TSMC, are able to produce the most
                                    advanced chips using ASML's technologies. In 2020, the
                                    
                                        Trump Administration successfully lobbied
                                    
                                    the Dutch government to prevent the sale of EUV systems to PRC companies.
                                

                                
                                    Hikvision has denied that it is seeking to make the acquisition.
                                

                                Research and Development
                                

                                
                                    Medical team implants an AI chip into a patient's brain to control epilepsy
                                    as part of a clinical trial. A medical team from Xuanwu Hospital in Beijing
                                    
                                        successfully implanted an AI chip
                                    
                                    into the brain of a patient diagnosed with bilateral temporal lobe
                                    epilepsy. The ""closed-loop brain-computer interface neurostimulation
                                    system"" prevents seizures by monitoring the patient's brain waves and
                                    producing an electrical signal at the onset of a seizure to regulate
                                    brainwave activity. According to reports, this was the first successful
                                    phase 3 clinical implantation for a procedure of this kind in China.
                                    However, the clinical trial is not necessarily novel. Researchers from
                                    Thomas Jefferson University Hospital in Philadelphia
                                    
                                        conducted similar trials
                                    
                                    in 2007.
                                

                                
                                
                                 
                                 
                                 
                                



                            
 

                             
                                
                                    About the China AI and Autonomy Report


                                    As world leaders grapple with how to respond to the People’s Republic of China’s (PRC) influence across the globe, understanding the PRC’s growing use of artificial 
                                        intelligence and autonomous systems to further its national security, economic, and political objectives will become increasingly important. 
                                        The China AI and Autonomy Report, a biweekly newsletter published by CNA, provides important news, developments, and policies regarding artificial 
                                        intelligence (AI) and autonomy in the PRC. We welcome your questions, comments, or subscription requests at chinaai@cna.org.

                                    
                                
 
                                
                                    
                                        The PLA and Intelligent Warfare: A Preliminary Analysis
                                        
                                    
                                    This paper explores the PLA concept of intelligent warfare, according to articles by
                                        writers from the People's Republic of China (PRC). It concludes that PRC discussions of
                                        intelligent warfare reflect an ongoing debate over the nature and effects of the
                                        widespread use of AI and autonomy in warfare.

                                

                                
    China AI and Autonomy Report Archive

        	Issue 1: November 2, 2021 [ web | pdf ]
	Issue 2: November 9, 2021 [ web | pdf ]
	Issue 3: November 18, 2021 [ web | pdf ]
	Issue 4: December 2, 2021 [ web | pdf ]
	Issue 5: December 16, 2021 [ web | pdf ]
	Issue 6: January 13, 2022 [ web | pdf ]
	Issue 7: January 27, 2022 [ web | pdf ]
	Issue 8: February 10, 2022 [ web | pdf ]
	Issue 9: February 24, 2022 [ web | pdf ]
	Issue 10: March 10, 2022 [ web | pdf ]
	Issue 11: March 24, 2022 [ web | pdf ]
	Issue 12: April 7, 2022 [ web | pdf ]
	Issue 13: April 21, 2022 [ web | pdf ]






    
    
    
    Center for Autonomy and AI
    
    

    
    
    
    Artificial Intelligence and Autonomy in Russia
    
    

    




                        
                            
 
                        
 
                        
                        
                

                

            

        

    


    
	    
    
        
            
				Contact Us

	
				Site Information

				
				Careers


				
                                            
						
							

							
								       
								  
										3003 Washington Boulevard
									
										Arlington, VA
										22201

										USA


										For additional contact information, click here.													
								
							

							
								  
									703.824.2000

									General Inquiries

									Media Inquiries
								
							

							
								About This Site

								   Privacy Policy

								   Employee Information
								
	
							

							
								Careers


								Have questions about recruiting and/or employment? Please contact inquiriesjobs@cna.org.

								CNA is an Equal Opportunity/Affirmative Action Employer, committed to diversity in the workplace.
									
							

                        

					
						
							

							
								       
								  
										3003 Washington Boulevard
									
										Arlington, VA
										22201

										USA


								
							

							
								  
									703.824.2000
								
									General Inquiries

									Media Inquiries
												
								
								For additional contact information, click here.
								 

							

							
								   Site Information


								   About This Site

								   Privacy Policy

								   Employee Information
								
	
							

							
								Careers

								Have questions about recruiting and/or employment? Please contact inquiriesjobs@cna.org.

								CNA is an Equal Opportunity/Affirmative Action Employer, committed to diversity in the workplace.

							

						

                    
            
                
                
            
        
        

        
            
                
                    Copyright © 2021 CNA All Rights Reserved
                    	
	
	
	
	
	


				

            

        

    

	
        
    
    
     
     
    
    
    
    
	 
    
	
	
    	
	
    	
	
"""

In [6]:
import re

In [12]:
def cleaning(text):
      #cleaned_text = text.lower() # lower case
      cleaned_text = text.replace("\n", " ")
      cleaned_text = cleaned_text.replace("\t", " ")
      cleaned_text = cleaned_text.replace('\r', '')

      cleaned_text = re.sub('[^\S\r\n]{2,}', ' ', cleaned_text) # extra spaces
      cleaned_text = cleaned_text.rstrip()
      
      return cleaned_text

In [13]:
test2 =  cleaning(test)

In [14]:
test2

'The China AI and Autonomy Report | CNA Contact Us Directions Toggle navigation About Us Our Experts Our Research Centers and Divisions Center for Naval Analyses Institute for Public Research News News and Information InDepth Careers Apply Now Careers At CNA What We Offer Benefits Research Internship Program Diversity Community Video Podcasts CNA Talks AI with AI Coming in from the Cold Justice Talks Center for Naval Analyses > China and Indo-Pacific Security Affairs > China Studies The China AI and Autonomy Report A biweekly newsletter on AI and autonomy developments in China Issue 11, March 24, 2022 [Download PDF Version] Welcome to the China AI and Autonomy Report, a biweekly newsletter published by CNA. As the war in Ukraine continues, rumors have been circulating about the role of PRC-made drones in the conflict. In this issue we note two of them—allegations that Russia has asked the PRC to supply drones, and accusations that a leading PRC-based commercial drone manufacturer has b